In [160]:
import pandas as pd
import smtplib
import glob
import os
from email.mime.text import MIMEText
from email.utils import COMMASPACE
import datetime
import calendar

#基本情報の取得
#各フォルダへのアクセス
basic_path = os.path.join(r'C:\Users\homas\Desktop\python\samurai_提出済課題\vegetable_order\samples')
#各店舗からのオーダー取得
od_path = glob.glob(os.path.join(basic_path, 'order_new', '*.xlsx'))
#現在在庫の取得(inventory.xlsx)
in_path = glob.glob(os.path.join(basic_path, 'inventory', '*.xlsx'))
#しきい値、追加量の取得(pickup.xlsx)
up_path = glob.glob(os.path.join(basic_path, 'pickup', '*.xlsx'))

#現在在庫計算用のデータフレーム(inventory.xlsx)
in_df = pd.read_excel(in_path[0], usecols = "C:I")
#しきい値、追加量計算用のデータフレーム(pickup.xlsx)
up_df = pd.read_excel(up_path[0])
#在庫更新用データフレームの取得(inventory.xlsx)
out_df = pd.read_excel(in_path[0], usecols = "A:I")

# 現在の日付と曜日を取得
today = datetime.datetime.today()
weekday = today.weekday()
today_str = today.strftime("%Y-%m-%d %H:%M:%S")
# 曜日を文字列に変換
weekday_str = calendar.day_abbr[weekday]

#コード
# オーダー数を把握する関数
def order_sum(od_path):
    order = pd.concat([pd.read_excel(f) for f in od_path], axis=0)
    order = order.fillna(0)
    for col in order.columns:
        order[col] = order[col].astype('int64')
    return order.sum()
    
# 在庫表の最終行の情報を抽出
def inventory_info(in_df):
    for col in in_df.columns:
        in_df[col] = in_df[col].astype('int64')
    return latest_inventory = in_df.iloc[-1]

# オーダー数と在庫情報を取得
order = order_sum(od_path)
inv = inventory_info(in_df)

#差し引き後の現在在庫数
ttl = inv - order
print("現在の在庫数:")
print(ttl)
print("------------------")

# しきい値を確認
def low_stock_items(up_df, ttl):
    df = up_df.iloc[0, 1:8]
    df = df.astype("int64")
    # しきい値を下回った野菜の名前を抽出
    low_stock_items = df.loc[df > ttl].index.tolist()
    return low_stock_items


#しきい値を下回る野菜の抽出
low = low_stock_items(up_df, ttl)
print("しきい値を下回る野菜:")
print(low)

# 発注数を確認
def order_items(up_df, low):
    # しきい値を下回った野菜の名前を抽出
    order = up_df.loc[1,low]
    return order

order = order_items(up_df, low)
print("発注数")
print(order[0])

#現在在庫の更新
def inventory_update(ttl,out_df):
    #更新用データフレームの作成
    df1 = pd.DataFrame({'日付': [today_str],
                        '曜日': [weekday_str],
                        ttl.index[0]: [ttl[0]],
                        ttl.index[1]: [ttl[1]],
                        ttl.index[2]: [ttl[2]],
                        ttl.index[3]: [ttl[3]],
                        ttl.index[4]: [ttl[4]],
                        ttl.index[5]: [ttl[5]],
                        ttl.index[6]: [ttl[6]],
                        },)

    out_df = pd.concat([out_df,df1],ignore_index=True,)
    return out_df

update = inventory_update(ttl,out_df)
# 在庫表を保存
update.to_excel(in_path[0], index=False)

password = "**** **** **** ****"
#発注メールの送信
def send_mail(password):
    # メールの送信先
    to_addr = ["farmer@gmail.com"]
    
    # メールの件名
    subject = "野菜発注のご連絡"
    
    # メールの本文
    body = "下記の野菜を発注いたします。\n\n野菜名: {}\n発注数: {}\n".format(low, order[0])
    
    # メッセージの作成
    msg = MIMEText(body)
    msg['From'] = "super.market@gmail.com"
    msg['To'] = COMMASPACE.join(to_addr)
    msg['Subject'] = subject
    
    # GmailのSMTPサーバーへの接続
    smtp_server = smtplib.SMTP("smtp.gmail.com", 587)
    smtp_server.starttls()
    
    # 認証
    smtp_server.login("super.market@gmail.com", password)
    
    # メールの送信
    smtp_server.send_message(msg)
    
    # SMTPサーバーの切断
    smtp_server.quit()

send_mail(password)

現在の在庫数:
トマト      60
キャベツ     52
レタス      61
白菜       59
ほうれん草    52
大根       33
ニンジン     18
dtype: int64
------------------
しきい値を下回る野菜:
['ニンジン']
発注数
80


In [ ]:
# inventryを読み込むDataframe化する in_df ファイル名も引数で渡す

# pickupを読み込んでDataframe化する pu_df ファイル名も引数で渡す

# grasp_orderの引数をin_dfにする

# low_stock_itemsの引数をpu_dfにする

# low_stock_itemsの引数をpu_dfにして、再読み込みをしなくする
# ttlをもう一回使う必要もない
# low_stock_vegetablesを引数にして
# "ニンジン"の行の発注量の列の数値のみを取り出す
# Dataframeに列追加して返す
#[ニンジン | 80]